In [ ]:
! pip install spacy

In [ ]:
! python -m spacy download en_core_web_sm

In [ ]:
import spacy
from spacy.gold import iob_to_biluo
nlp = spacy.load("en_core_web_sm", disable = ['textcat'])

## Raw Text To Annotation Job Using Spacy 

In [ ]:
stop_list = ["will", "shall", "would", "should", "shouldn't", "ought", "must", "mustn’t", "may", "might", "can", "could","have", "has", "don’t", "seemed", "seems", "seem", "tried"]
allowed_forms = ["NE", "VERB", "NOUN", "PRON", "PROPN"]#, "ADJ",]
possesive_determiners = ["my",  "his", "their", "her", "our", "your", "its"]

data = {
    "tokens": [], # {i, document, pos, lemma, text }
    "candidateMentions":[]
} # topics, available_mentions 
for index, topic in topics:
    for doc_no in topic:
        doc = nlp(topic[doc_no])
        iobs = ['{}-{}'.format(i.ent_iob_, i.ent_type_) if i.ent_iob_ != 'O' else i.ent_iob_ for i in doc]
        biluo = iob_to_biluo(iobs)
        startNE = None 
        for i, t in enumerate(doc):
            print(biluo[i] + "-" + t.text)
            if startNE is None:
                if biluo[i][0] == "B":
                    if t.pos_ == "DET":
                        startNE = len(data["tokens"])  + 1
                    else:
                        startNE = len(data["tokens"])
                elif (t.pos_ in allowed_forms and (t.text.lower() not in stop_list) and (t.text.lower() not in possesive_determiners)) or biluo[i][0] == "U" : 
                    data["candidateMentions"].append({
                        "start":len(data["tokens"]),
                        "end":len(data["tokens"])
                        })
            elif biluo[t.i][0] == "L":
                data["candidateMentions"].append(
                    {"start":startNE,
                    "end":len(data["tokens"])})
                startNE = None

            data["tokens"].append({"i": len(data["tokens"]),
                            "pos":t.pos_,
                            'document':doc_no,
                            'text':t.text, 
                            'lemma': t.lemma_,
                            'noWhite': not t.whitespace_
                            })
firstMention = data["candidateMentions"][0]
firstMention["clustId"] = "{}-{}".format(firstMention["start"], firstMention["end"])
data["selectedCluster"] = "{}-{}".format(firstMention["start"], firstMention["end"])
data["mentions"] = data["candidateMentions"]
data["fixableSpans"] = True
data["newMentions"] = True
data["reassignable"] = True
data["mode"] =  "annotation"


## JSON Config

In [ ]:
import json

In [ ]:
with open('annotation.json', 'w') as outfile:
    json.dump(data, outfile)

## MTURK CSV

In [ ]:
# HTML Escape
output_csv = ""
data = html.escape(json.dumps(data))
df = pd.DataFrame([{"data":data}]).to_csv(output_csv, index=False)